In [11]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np

In [3]:
df_user_log = pd.read_csv('https://drive.google.com/uc?id=1DwEvKOYl0hQmd2muwDa13O17smPPgn0D')
df_user_info = pd.read_csv('https://drive.google.com/uc?id=1wM9LY4rAsMLGe6GUZNQd50R8WHiH9xaf')

In [5]:
userTrain = pd.read_csv(r"C:\Users\030879522\OneDrive - csulb\Documents\Projects\PR_Final\CECS550_FinalProject\train_format1.csv")

In [6]:
df_user_log.isnull().any()
df_user_info.isnull().any()
user_action = df_user_log.groupby('user_id').size().to_frame().reset_index()
user_action.columns = ['user_id','user_total_action']
user_action.head()

,user_id,user_total_action
0,192,1
1,289,1
2,436,3
3,445,1
4,616,1


In [9]:
user_gender = df_user_info[['user_id','gender']]
user_gender = user_gender.fillna(2)
user_gender.head()

,user_id,gender
0,376517,1.0
1,234512,0.0
2,344532,0.0
3,186135,0.0
4,30230,0.0


In [16]:
user_activity = df_user_log[['user_id','action_type']].copy()
user_activity_info = label_binarize(np.array(user_activity.action_type), classes=[0,1,2,3])
user_activity_info = pd.DataFrame(data = user_activity_info, columns=['Click','add_to_cart','purchase','add_to_favorite'])
user_activity['user_Click']= user_activity_info.Click
user_activity['user_Add_to_cart']=user_activity_info.add_to_cart
user_activity['user_purchase']=user_activity_info.purchase
user_activity['user_Add_to_favorite']=user_activity_info.add_to_favorite
user_activity_count = user_activity.groupby('user_id').agg('sum').reset_index()
user_activity_count.columns = ['user_id','action_type','user_Click_Count','user_Add_to_cart_count','user_purchase_count','user_Add_to_favorite_count']
user_activity_count = user_activity_count.drop(columns=['action_type'])
user_activity_count.head()

,user_id,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count
0,192,1,0,0,0
1,289,1,0,0,0
2,436,2,0,0,1
3,445,1,0,0,0
4,616,1,0,0,0


In [17]:
user_age = df_user_info[['user_id','age_range']].copy()
user_age = user_age.fillna(9)
user_age = user_age.replace(8,7)
user_age = user_age.drop_duplicates()
user_age_info = label_binarize(np.array(user_age.age_range),classes=[1,2,3,4,5,6,7,9])
user_age_info = pd.DataFrame(data= user_age_info, columns = ['age_0_18','age_18_24','age_25_29','age_30_34','age_35_39','age_40_49','age_50_','age_unknown'])
user_age['user_age_0_18']=user_age_info['age_0_18']
user_age['user_age_18_24']=user_age_info['age_18_24']
user_age['user_age_25_29']=user_age_info['age_25_29']
user_age['user_age_30_34']=user_age_info['age_30_34']
user_age['user_age_35_39']=user_age_info['age_35_39']
user_age['user_age_40_49']=user_age_info['age_40_49']
user_age['user_age_50_']=user_age_info['age_50_']
user_age['user_age_unknown']=user_age_info['age_unknown']
user_age = user_age.drop(columns=['age_range'])
user_age.head()

,user_id,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,user_age_50_,user_age_unknown
0,376517,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,234512,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,344532,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,186135,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,30230,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,user_id,seller_id,user_seller_item_count
0,192,1986,1
1,289,1728,1
2,436,1274,1
3,445,840,1
4,616,2101,1


In [19]:
#user level feature
#total_Actions, user_activity
#gender, age_range
user_level_feature = pd.merge(user_gender, user_age, on=['user_id'],how='left')
#add user activity
user_level_feature = pd.merge(user_level_feature, user_activity_count, on=['user_id'],how='left')
#add user total actions
user_level_feature = pd.merge(user_level_feature, user_action, on=['user_id'],how='left')
user_level_feature.head()

,user_id,gender,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,user_age_50_,user_age_unknown,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count,user_total_action
0,376517,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,234512,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,344532,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,186135,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,30230,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [20]:
user_seller_item = df_user_log[['user_id','seller_id','item_id']].copy()
user_seller_item.drop_duplicates(inplace=True)
user_seller_item = user_seller_item.groupby(['user_id','seller_id']).size().reset_index()
user_seller_item.columns = ['user_id','seller_id','user_seller_item_count']
user_seller_item.head()

,user_id,seller_id,user_seller_item_count
0,192,1986,1
1,289,1728,1
2,436,1274,1
3,445,840,1
4,616,2101,1


In [21]:
user_seller_cat = df_user_log[['user_id','seller_id','cat_id']].copy()
user_seller_cat.drop_duplicates(inplace=True)
user_seller_cat = user_seller_cat.groupby(['user_id','seller_id']).size().reset_index()
user_seller_cat.columns = ['user_id','seller_id','user_seller_category_count']
user_seller_cat.head()

,user_id,seller_id,user_seller_category_count
0,192,1986,1
1,289,1728,1
2,436,1274,1
3,445,840,1
4,616,2101,1


In [22]:
#user-seller interaction level feature
#add user-seller item count, category count
user_seller_level_feature = pd.merge(user_seller_item, user_seller_cat, on=['user_id','seller_id'],how='left')
#add user-seller brand count
# user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_brand, on=['user_id','seller_id'],how='left')
#add user-seller activity
# user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_item_act_count, on=['user_id','seller_id'],how='left')
#add user-seller active days by Month
# user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_Month_counts, on=['user_id','seller_id'],how='left')
#add user-seller active days total
# user_seller_level_feature = pd.merge(user_seller_level_feature, user_seller_days, on=['user_id','seller_id'],how='left') 
user_seller_level_feature.head()

,user_id,seller_id,user_seller_item_count,user_seller_category_count
0,192,1986,1,1
1,289,1728,1,1
2,436,1274,1,1
3,445,840,1,1
4,616,2101,1,1


In [25]:
#combine train data and new added feature
#add user-level feature
train_complete = pd.merge(userTrain, user_level_feature, on=['user_id'],how='left')
#add user-seller level feature
user_seller_level_feature = user_seller_level_feature.rename(columns = {'seller_id':'merchant_id'})
train_complete = pd.merge(train_complete, user_seller_level_feature, on=['user_id','merchant_id'],how='left')
train_complete.head()

,user_id,merchant_id,label,gender,user_age_0_18,user_age_18_24,user_age_25_29,user_age_30_34,user_age_35_39,user_age_40_49,user_age_50_,user_age_unknown,user_Click_Count,user_Add_to_cart_count,user_purchase_count,user_Add_to_favorite_count,user_total_action,user_seller_item_count,user_seller_category_count
0,34176,3906,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34176,121,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34176,4356,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34176,2217,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,230784,4818,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
